In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_predict_data()
Predictor.update_factors()

Update Files
Thu Aug 29 02:02:23 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.02 Secs
Thu Aug 29 02:02:23 2024 : Updated ~ DB_information\description.feather Done! Cost 0.04 Secs
Thu Aug 29 02:02:23 2024 : Updated ~ DB_information\st.feather Done! Cost 0.01 Secs
Thu Aug 29 02:02:25 2024 : Updated ~ DB_information\industry.feather Done! Cost 2.52 Secs
Thu Aug 29 02:02:30 2024 : Updated ~ DB_information\concepts.feather Done! Cost 4.58 Secs
Thu Aug 29 02:02:31 2024 : Updated ~ DB_models\longcl_exp\2024\longcl_exp.20240827.feather Done! Cost 0.13 Secs
Thu Aug 29 02:02:31 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20240828.feather Done! Cost 0.04 Secs
Thu Aug 29 02:02:31 2024 : Updated ~ DB_models\risk_cov\2024\risk_cov.20240828.feather Done! Cost 0.03 Secs
Thu Aug 29 02:02:31 2024 : Updated ~ DB_models\risk_spec\2024\risk_spec.20240828.feather Done! Cost 0.00 Secs
Thu Aug 29 02:02:33 2024 : Updated ~ DB_trade\day\2024\day.20240828.feather Done! Cost 0.05 Secs
Thu A

FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Coding\\learndl\\learndl\\models\\gru_day\\default.yaml'

In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [10]:
# DataAPI.prepare_predict_data()
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [1]:
from src.api import DataAPI , Predictor
# DataAPI.prepare_predict_data()
md = Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [ ]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
